In [42]:
#import libraries
import xgboost as xgb
import itertools
from prophet import Prophet
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("D:/Forecasting Models and Time Series for Business in Python/Daily Bike Sharing.csv")
data.head()

instant      dteday  season  yr  mnth  holiday  weekday  workingday  \
0        1  2011-01-01       1   0     1        0        6           0   
1        2  2011-01-02       1   0     1        0        0           0   
2        3  2011-01-03       1   0     1        0        1           1   
3        4  2011-01-04       1   0     1        0        2           1   
4        5  2011-01-05       1   0     1        0        3           1   

   weathersit      temp     atemp       hum  windspeed  casual  registered  \
0           2  0.344167  0.363625  0.805833   0.160446     331         654   
1           2  0.363478  0.353739  0.696087   0.248539     131         670   
2           1  0.196364  0.189405  0.437273   0.248309     120        1229   
3           1  0.200000  0.212122  0.590435   0.160296     108        1454   
4           1  0.226957  0.229270  0.436957   0.186900      82        1518   

    cnt  
0   985  
1   801  
2  1349  
3  1562  
4  1600

In [32]:
#select variables
dataset = data.loc[:, ["dteday","cnt", "holiday", "workingday", "weathersit",
                       "temp", "atemp", "hum", "windspeed"]]
dataset.head()                       

dteday   cnt  holiday  workingday  weathersit      temp     atemp  \
0  2011-01-01   985        0           0           2  0.344167  0.363625   
1  2011-01-02   801        0           0           2  0.363478  0.353739   
2  2011-01-03  1349        0           1           1  0.196364  0.189405   
3  2011-01-04  1562        0           1           1  0.200000  0.212122   
4  2011-01-05  1600        0           1           1  0.226957  0.229270   

        hum  windspeed  
0  0.805833   0.160446  
1  0.696087   0.248539  
2  0.437273   0.248309  
3  0.590435   0.160296  
4  0.436957   0.186900

In [33]:
#Date variable
dataset.dteday = pd.to_datetime(dataset.dteday,
                                format = "%Y-%m-%d")
dataset.dteday

0     2011-01-01
1     2011-01-02
2     2011-01-03
3     2011-01-04
4     2011-01-05
         ...    
726   2012-12-27
727   2012-12-28
728   2012-12-29
729   2012-12-30
730   2012-12-31
Name: dteday, Length: 731, dtype: datetime64[ns]

In [34]:
#renaming variable
dataset = dataset.rename(columns = {'cnt' : 'y'})
dataset = dataset.rename(columns = {'dteday' : 'ds'})
dataset.head()

ds     y  holiday  workingday  weathersit      temp     atemp  \
0 2011-01-01   985        0           0           2  0.344167  0.363625   
1 2011-01-02   801        0           0           2  0.363478  0.353739   
2 2011-01-03  1349        0           1           1  0.196364  0.189405   
3 2011-01-04  1562        0           1           1  0.200000  0.212122   
4 2011-01-05  1600        0           1           1  0.226957  0.229270   

        hum  windspeed  
0  0.805833   0.160446  
1  0.696087   0.248539  
2  0.437273   0.248309  
3  0.590435   0.160296  
4  0.436957   0.186900

In [35]:
holiday_dates = dataset[dataset.holiday == 1].ds
holidays = pd.DataFrame({'holiday' : 'holi',
                         'ds': pd.to_datetime(holiday_dates),
                         'lower_window': -3,
                         'upper_window': 1})
holidays.head()

holiday         ds  lower_window  upper_window
16     holi 2011-01-17            -3             1
51     holi 2011-02-21            -3             1
104    holi 2011-04-15            -3             1
149    holi 2011-05-30            -3             1
184    holi 2011-07-04            -3             1

In [36]:
#removing holiday column
dataset = dataset.drop(columns = "holiday")
dataset.head(1)

ds    y  workingday  weathersit      temp     atemp       hum  \
0 2011-01-01  985           0           2  0.344167  0.363625  0.805833   

   windspeed  
0   0.160446

In [37]:
#Training and test set
test_days = 31
training_set = dataset.iloc[:-test_days, :]
test_set = dataset.iloc[-test_days:, :]
test_set.tail()

ds     y  workingday  weathersit      temp     atemp       hum  \
726 2012-12-27  2114           1           2  0.254167  0.226642  0.652917   
727 2012-12-28  3095           1           2  0.253333  0.255046  0.590000   
728 2012-12-29  1341           0           2  0.253333  0.242400  0.752917   
729 2012-12-30  1796           0           1  0.255833  0.231700  0.483333   
730 2012-12-31  2729           1           2  0.215833  0.223487  0.577500   

     windspeed  
726   0.350133  
727   0.155471  
728   0.124383  
729   0.350754  
730   0.154846

In [38]:
#Facebook Prophet model
m = Prophet(growth = "linear",
            yearly_seasonality = True,
            weekly_seasonality = True,
            daily_seasonality = False,
            holidays = holidays,
            seasonality_mode = "multiplicative",
            seasonality_prior_scale = 20,
            holidays_prior_scale = 20,
            changepoint_prior_scale = 0.01)
m.add_regressor('workingday')
m.add_regressor('weathersit')
m.add_regressor('temp')
m.add_regressor('atemp')
m.add_regressor('hum')
m.add_regressor('windspeed')
m.fit(training_set)

18:12:20 - cmdstanpy - INFO - Chain [1] start processing
18:12:20 - cmdstanpy - INFO - Chain [1] done processing


In [39]:
#Create Future Dataframe
future = m.make_future_dataframe(periods = len(test_set),
                                 freq = "D")
future.tail()

ds
726 2012-12-27
727 2012-12-28
728 2012-12-29
729 2012-12-30
730 2012-12-31

In [40]:
#merge regressors
future = pd.concat([future, dataset.iloc[:,2:]],
                   axis = 1)
future.tail()

ds  workingday  weathersit      temp     atemp       hum  \
726 2012-12-27           1           2  0.254167  0.226642  0.652917   
727 2012-12-28           1           2  0.253333  0.255046  0.590000   
728 2012-12-29           0           2  0.253333  0.242400  0.752917   
729 2012-12-30           0           1  0.255833  0.231700  0.483333   
730 2012-12-31           1           2  0.215833  0.223487  0.577500   

     windspeed  
726   0.350133  
727   0.155471  
728   0.124383  
729   0.350754  
730   0.154846

In [41]:
#forecast
forecast = m.predict(future)
forecast.head()

ds        trend  yhat_lower   yhat_upper  trend_lower  trend_upper  \
0 2011-01-01  2429.776904  274.639879  2164.623680  2429.776904  2429.776904   
1 2011-01-02  2435.398244   52.781019  1982.814006  2435.398244  2435.398244   
2 2011-01-03  2441.019585  370.166907  2254.347723  2441.019585  2441.019585   
3 2011-01-04  2446.640925  450.453198  2305.364375  2446.640925  2446.640925   
4 2011-01-05  2452.262265  639.018538  2532.923198  2452.262265  2452.262265   

      atemp  atemp_lower  atemp_upper  extra_regressors_multiplicative  ...  \
0 -0.055503    -0.055503    -0.055503                        -0.279363  ...   
1 -0.060175    -0.060175    -0.060175                        -0.288137  ...   
2 -0.137830    -0.137830    -0.137830                        -0.188039  ...   
3 -0.127095    -0.127095    -0.127095                        -0.177352  ...   
4 -0.118992    -0.118992    -0.118992                        -0.111656  ...   

   workingday  workingday_lower  workingday_upper    yearly  yearly_lower  \
0      0.0000            0.0000            0.0000 -0.261834     -0.261834   
1      0.0000            0.0000            0.0000 -0.255753     -0.255753   
2      0.0333            0.0333            0.0333 -0.248428     -0.248428   
3      0.0333            0.0333            0.0333 -0.239970     -0.239970   
4      0.0333            0.0333            0.0333 -0.230511     -0.230511   

   yearly_upper  additive_terms  additive_terms_lower  additive_terms_upper  \
0     -0.261834             0.0                   0.0                   0.0   
1     -0.255753             0.0                   0.0                   0.0   
2     -0.248428             0.0                   0.0                   0.0   
3     -0.239970             0.0                   0.0                   0.0   
4     -0.230511             0.0                   0.0                   0.0   

          yhat  
0  1256.244173  
1  1010.113438  
2  1312.884965  
3  1421.956309  
4  1599.486159  

[5 rows x 46 columns]

In [43]:
#XGBoost

prophet_variables = forecast.loc[:, ["trend", "holi", "weekly", "yearly"]]
df_xgb = pd.concat([dataset, prophet_variables], axis = 1)
df_xgb.head()

ds     y  workingday  weathersit      temp     atemp       hum  \
0 2011-01-01   985           0           2  0.344167  0.363625  0.805833   
1 2011-01-02   801           0           2  0.363478  0.353739  0.696087   
2 2011-01-03  1349           1           1  0.196364  0.189405  0.437273   
3 2011-01-04  1562           1           1  0.200000  0.212122  0.590435   
4 2011-01-05  1600           1           1  0.226957  0.229270  0.436957   

   windspeed        trend  holi    weekly    yearly  
0   0.160446  2429.776904   0.0  0.058217 -0.261834  
1   0.248539  2435.398244   0.0 -0.041346 -0.255753  
2   0.248309  2441.019585   0.0 -0.025690 -0.248428  
3   0.160296  2446.640925   0.0 -0.001490 -0.239970  
4   0.186900  2452.262265   0.0 -0.005584 -0.230511

In [44]:
#Training and test set
test_days = 31
training_set = df_xgb.iloc[:-test_days, :]
test_set = df_xgb.iloc[-test_days:, :]
test_set.tail()

ds     y  workingday  weathersit      temp     atemp       hum  \
726 2012-12-27  2114           1           2  0.254167  0.226642  0.652917   
727 2012-12-28  3095           1           2  0.253333  0.255046  0.590000   
728 2012-12-29  1341           0           2  0.253333  0.242400  0.752917   
729 2012-12-30  1796           0           1  0.255833  0.231700  0.483333   
730 2012-12-31  2729           1           2  0.215833  0.223487  0.577500   

     windspeed        trend  holi    weekly    yearly  
726   0.350133  6403.273056   0.0  0.003053 -0.272033  
727   0.155471  6408.712615   0.0  0.012840 -0.272269  
728   0.124383  6414.152175   0.0  0.058217 -0.271072  
729   0.350754  6419.591735   0.0 -0.041346 -0.268434  
730   0.154846  6425.031294   0.0 -0.025690 -0.264379

In [45]:
#isolate X and Y
y_train = training_set.y
y_test = test_set.y
X_train = training_set.iloc[:, 2:]
X_test = test_set.iloc[:, 2:]

In [46]:
#create XGBoost Matrices
Train = xgb.DMatrix(data = X_train, label = y_train)
Test = xgb.DMatrix(data = X_test, label = y_test)

In [50]:
#Set the parameters
parameters = {'learning_rate': 0.1,
              'max_depth': 3,
              'colsample_bytree': 1,
              'subsample': 1,
              'min_child_weight': 1,
              'gamma': 1,
              'random_state': 1502,
              'eval_metric': "rmse",
              'objective': "reg:squarederror"}

In [52]:
#XGBoost Model
model = xgb.train(params = parameters,
                  dtrain = Train,
                  num_boost_round = 100,
                  evals = [(Test, "y")])

[0]	y-rmse:3891.58894
[1]	y-rmse:3563.69059
[2]	y-rmse:3274.75734
[3]	y-rmse:2950.62380
[4]	y-rmse:2716.06266
[5]	y-rmse:2522.39072
[6]	y-rmse:2317.96280
[7]	y-rmse:2156.40260
[8]	y-rmse:2024.56200
[9]	y-rmse:1934.05555
[10]	y-rmse:1866.22580
[11]	y-rmse:1791.30150
[12]	y-rmse:1740.37022
[13]	y-rmse:1686.21001
[14]	y-rmse:1645.00477
[15]	y-rmse:1607.49185
[16]	y-rmse:1550.18027
[17]	y-rmse:1505.37005
[18]	y-rmse:1497.57975
[19]	y-rmse:1468.93298
[20]	y-rmse:1447.49851
[21]	y-rmse:1470.55166
[22]	y-rmse:1458.53921
[23]	y-rmse:1450.04251
[24]	y-rmse:1436.79017
[25]	y-rmse:1437.45231
[26]	y-rmse:1434.34955
[27]	y-rmse:1426.79344
[28]	y-rmse:1405.47873
[29]	y-rmse:1382.48373
[30]	y-rmse:1368.36629
[31]	y-rmse:1372.42708
[32]	y-rmse:1364.42167
[33]	y-rmse:1354.23317
[34]	y-rmse:1351.62577
[35]	y-rmse:1345.77694
[36]	y-rmse:1345.84280
[37]	y-rmse:1341.13245
[38]	y-rmse:1338.39152
[39]	y-rmse:1326.23727
[40]	y-rmse:1326.54284
[41]	y-rmse:1325.68060
[42]	y-rmse:1313.05545
[43]	y-rmse:1304.0738

In [53]:
#Forecasting
predictions_xgb = pd.Series(model.predict(Test), name = "XGBoost")
predictions_xgb.index = test_set.ds
predictions_xgb[:2]

ds
2012-12-01    3996.192383
2012-12-02    3068.570312
Name: XGBoost, dtype: float32

In [56]:
#set up index
training_set.index = training_set.ds
test_set.index = test_set.ds

ds     y  workingday  weathersit      temp     atemp  \
ds                                                                        
2012-12-01 2012-12-01  5191           0           2  0.298333  0.316904   
2012-12-02 2012-12-02  4649           0           2  0.347500  0.359208   
2012-12-03 2012-12-03  6234           1           1  0.452500  0.455796   
2012-12-04 2012-12-04  6606           1           1  0.475833  0.469054   
2012-12-05 2012-12-05  5729           1           1  0.438333  0.428012   
2012-12-06 2012-12-06  5375           1           1  0.255833  0.258204   
2012-12-07 2012-12-07  5008           1           2  0.320833  0.321958   
2012-12-08 2012-12-08  5582           0           2  0.381667  0.389508   
2012-12-09 2012-12-09  3228           0           2  0.384167  0.390146   
2012-12-10 2012-12-10  5170           1           2  0.435833  0.435575   
2012-12-11 2012-12-11  5501           1           2  0.353333  0.338363   
2012-12-12 2012-12-12  5319           1           2  0.297500  0.297338   
2012-12-13 2012-12-13  5532           1           1  0.295833  0.294188   
2012-12-14 2012-12-14  5611           1           1  0.281667  0.294192   
2012-12-15 2012-12-15  5047           0           1  0.324167  0.338383   
2012-12-16 2012-12-16  3786           0           2  0.362500  0.369938   
2012-12-17 2012-12-17  4585           1           2  0.393333  0.401500   
2012-12-18 2012-12-18  5557           1           1  0.410833  0.409708   
2012-12-19 2012-12-19  5267           1           1  0.332500  0.342162   
2012-12-20 2012-12-20  4128           1           2  0.330000  0.335217   
2012-12-21 2012-12-21  3623           1           2  0.326667  0.301767   
2012-12-22 2012-12-22  1749           0           1  0.265833  0.236113   
2012-12-23 2012-12-23  1787           0           1  0.245833  0.259471   
2012-12-24 2012-12-24   920           1           2  0.231304  0.258900   
2012-12-25 2012-12-25  1013           0           2  0.291304  0.294465   
2012-12-26 2012-12-26   441           1           3  0.243333  0.220333   
2012-12-27 2012-12-27  2114           1           2  0.254167  0.226642   
2012-12-28 2012-12-28  3095           1           2  0.253333  0.255046   
2012-12-29 2012-12-29  1341           0           2  0.253333  0.242400   
2012-12-30 2012-12-30  1796           0           1  0.255833  0.231700   
2012-12-31 2012-12-31  2729           1           2  0.215833  0.223487   

                 hum  windspeed        trend      holi    weekly    yearly  
ds                                                                          
2012-12-01  0.806667   0.059704  6261.844503  0.000000  0.058217 -0.138118  
2012-12-02  0.823333   0.124379  6267.284063  0.000000 -0.041346 -0.137838  
2012-12-03  0.767500   0.082721  6272.723623  0.000000 -0.025690 -0.137884  
2012-12-04  0.733750   0.174129  6278.163182  0.000000 -0.001490 -0.138389  
2012-12-05  0.485000   0.324021  6283.602742  0.000000 -0.005584 -0.139469  
2012-12-06  0.508750   0.174754  6289.042302  0.000000  0.003053 -0.141227  
2012-12-07  0.764167   0.130600  6294.481862  0.000000  0.012840 -0.143742  
2012-12-08  0.911250   0.101379  6299.921421  0.000000  0.058217 -0.147070  
2012-12-09  0.905417   0.157975  6305.360981  0.000000 -0.041346 -0.151243  
2012-12-10  0.925000   0.190308  6310.800541  0.000000 -0.025690 -0.156265  
2012-12-11  0.596667   0.296037  6316.240100  0.000000 -0.001490 -0.162114  
2012-12-12  0.538333   0.162937  6321.679660  0.000000 -0.005584 -0.168741  
2012-12-13  0.485833   0.174129  6327.119220  0.000000  0.003053 -0.176071  
2012-12-14  0.642917   0.131229  6332.558779  0.000000  0.012840 -0.184003  
2012-12-15  0.650417   0.106350  6337.998339  0.000000  0.058217 -0.192417  
2012-12-16  0.838750   0.100742  6343.437899  0.000000 -0.041346 -0.201170  
2012-12-17  0.907083   0.098258  6348.877459  0.000000 -0.025690 -0.210108  
2012-12-18  0.666250   0.221404  6354.317018  0.000000 -0.001490 -0.219059  
201

In [57]:
#Viz
training_set.y['2012-07-01':].plot(figsize = (9,6), legend = True)
test_set.y.plot(legend = True)
predictions_xgb.plot(legend = True)

<AxesSubplot: xlabel='ds'>

In [58]:
#MAE and RMSE
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(round(mean_absolute_error(test_set['y'], predictions_xgb),0))
print(round(np.sqrt(mean_squared_error(test_set['y'], predictions_xgb)), 0))

946.0
1174.0


In [59]:
#MAPE function
def MAPE(y_true, y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
MAPE(test_set['y'], predictions_xgb)

45.377004733275925